In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
import pdb
import seaborn as sns
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# measure = "GDT_TS"
# measure = "GDT_HA"
# measure = "qs_global"
measures = ["qs_global",
            #  "qs_best",
               "ics",
                 "ips",
                   "dockq_ave",
                     "tm_score", 
                     "lddt"]

# model = "first"
model = "best"
# mode = "easy"
# mode = "medium"
# mode = "hard"
mode = "all"
impute_value = -2
os.chdir('/home2/s439906/project/CASP16/oligomer/')
raw_path = "./group_by_target_EU_new/"
sum_df = None
for measure in measures:
    file = f"{measure}-{model}-{mode}-impute={impute_value}_raw.csv"
    df = pd.read_csv(raw_path + file, index_col=0)
    # remove the rows with more than 50% missing values
    df = df.dropna(thresh=df.shape[1] * 100 / 100)

    sum = df.sum(axis=1)
    # change the column name to measure
    sum.rename(measure, inplace=True)
    if sum_df is None:
        sum_df = sum
    else:
        sum_df = pd.concat([sum_df, sum], axis=1)
sum_df


In [ ]:
# get pairwise scatter plot
sns.pairplot(sum_df)

In [ ]:
result_dir = "/data/data1/conglab/jzhan6/CASP16/targetPDBs/Targets_oligo_interfaces_20240917/model_results/"
result_files = [file for file in os.listdir(result_dir) if file.startswith("T1") or file.startswith("H1")]
result_files

In [ ]:
data_all = None
for result_file in result_files:
    print("Processing {}".format(result_file))
    result_path = result_dir+ result_file
    data_tmp = pd.read_csv(result_path, sep="\t", index_col=0)
    data_tmp = data_tmp.replace("-", np.nan)
    if data_all is None:
        data_all = data_tmp
    else:
        data_all = pd.concat([data_all, data_tmp], axis=0)
# get the columns in measures
measures = ["qs_global",
            #  "qs_best",
               "ics",
                 "ips",
                   "dockq_ave",
                     "tm_score", 
                     "lddt"]
data_all = data_all[measures]
data_all.isna().sum()
# remove the rows with any nan
data_all = data_all.dropna()
# convert the data type to float
data_all = data_all.astype(float)

In [ ]:
sns.pairplot(data_all, plot_kws={'s': 10})


In [ ]:
# draw a correlation heatmap
corr = data_all.corr()
sns.heatmap(corr, annot=True, fmt=".2f", square=True, cmap='coolwarm')
plt.show()

In [ ]:

score_path = "./heatmap_interface/"
score_file = f"heatmap_custom_{model}_{mode}_impute={impute_value}_equal_weight.csv"
df = pd.read_csv(score_path + score_file, index_col=0)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def correlation_scatter_pairplot(data):
    # 创建 PairGrid 对象
    g = sns.PairGrid(data)
    
    # 下三角部分绘制散点图
    g.map_lower(sns.scatterplot)
    
    # 上三角部分绘制相关系数
    def corrplot(x, y, **kws):
        corr = np.corrcoef(x, y)[0, 1]
        ax = plt.gca()
        ax.annotate(f'{corr:.2f}', 
                    xy=(0.5, 0.5), 
                    xycoords=ax.transAxes,
                    ha='center', 
                    va='center',
                    fontsize=12,
                    fontweight='bold',
                    color='red')
    
    g.map_upper(corrplot)
    
    # 对角线部分绘制直方图
    g.map_diag(sns.histplot, kde=True)

# 使用示例
correlation_scatter_pairplot(data_all)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def correlation_scatter_pairplot(data):
    # 创建 PairGrid 对象
    g = sns.PairGrid(data)
    
    # 下三角部分绘制散点图
    g.map_lower(sns.scatterplot)
    
    # 上三角部分绘制相关系数
    def corrplot(x, y, **kws):
        corr = np.corrcoef(x, y)[0, 1]
        ax = plt.gca()
        ax.annotate(f'{corr:.2f}', 
                    xy=(0.5, 0.5), 
                    xycoords=ax.transAxes,
                    ha='center', 
                    va='center',
                    fontsize=12,
                    fontweight='bold',
                    color='red')
    
    g.map_upper(corrplot)
    
    # 对角线部分绘制分布直方图
    g.map_diag(sns.distplot, kde=True)

# 使用示例
correlation_scatter_pairplot(data_all)
